# Exploratory Data Analysis - Radio Signal Anomaly Detection

**Goal**: Build strong intuition about the data before applying any model.

We'll explore:
1. Dataset structure and basic statistics
2. Class and SNR distributions
3. Time domain visualization (IQ signals)
4. Frequency domain analysis (FFT, spectrograms)
5. Signal statistics (power, variance, amplitude)
6. Comparison between known classes (0-5) and anomalies (6-8)
7. SNR impact on signals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal as scipy_signal
from scipy.fft import fft, fftfreq, fftshift
import pandas as pd

from data_utils import load_train_data, load_test_anomalies, create_binary_labels

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Color palettes
KNOWN_COLOR = 'tab:blue'
ANOMALY_COLOR = 'tab:red'
CLASS_COLORS = plt.cm.tab10(np.arange(9))

## 1. Load and Inspect Data

In [ ]:
# Load both datasets
train_signals, train_labels, train_snr = load_train_data()
test_signals, test_labels, test_snr = load_test_anomalies()

print("=" * 60)
print("TRAINING DATA (classes 0-5 only)")
print("=" * 60)
print(f"Signals shape: {train_signals.shape}")
print(f"Signals dtype: {train_signals.dtype}")
print(f"Labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"SNR shape: {train_snr.shape}, dtype: {train_snr.dtype}")
print(f"Memory: {train_signals.nbytes / 1e6:.1f} MB")
print(f"\nSignal dimensions: {train_signals.shape[0]} samples x {train_signals.shape[1]} time steps x {train_signals.shape[2]} channels (I/Q)")

print("\n" + "=" * 60)
print("TEST DATA (classes 0-8, with 6-8 as anomalies)")
print("=" * 60)
print(f"Signals shape: {test_signals.shape}")
print(f"Labels shape: {test_labels.shape}")
print(f"SNR shape: {test_snr.shape}")
print(f"Memory: {test_signals.nbytes / 1e6:.1f} MB")

In [ ]:
# Basic signal statistics
print("\n" + "=" * 60)
print("SIGNAL VALUE RANGES")
print("=" * 60)
print(f"\nTraining signals:")
print(f"  Min: {train_signals.min():.6f}")
print(f"  Max: {train_signals.max():.6f}")
print(f"  Mean: {train_signals.mean():.6f}")
print(f"  Std: {train_signals.std():.6f}")

print(f"\nTest signals:")
print(f"  Min: {test_signals.min():.6f}")
print(f"  Max: {test_signals.max():.6f}")
print(f"  Mean: {test_signals.mean():.6f}")
print(f"  Std: {test_signals.std():.6f}")

## 2. Class and SNR Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training class distribution
ax = axes[0, 0]
unique_train, counts_train = np.unique(train_labels, return_counts=True)
bars = ax.bar(unique_train, counts_train, color=[CLASS_COLORS[i] for i in unique_train])
ax.set_xlabel('Class')
ax.set_ylabel('Count')
ax.set_title('Training Data: Class Distribution')
ax.set_xticks(unique_train)
for bar, count in zip(bars, counts_train):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, str(count), 
            ha='center', va='bottom', fontsize=10)

# Test class distribution
ax = axes[0, 1]
unique_test, counts_test = np.unique(test_labels, return_counts=True)
colors = [ANOMALY_COLOR if c >= 6 else KNOWN_COLOR for c in unique_test]
bars = ax.bar(unique_test, counts_test, color=colors)
ax.set_xlabel('Class')
ax.set_ylabel('Count')
ax.set_title('Test Data: Class Distribution (Red = Anomaly)')
ax.set_xticks(unique_test)
for bar, count in zip(bars, counts_test):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, str(count), 
            ha='center', va='bottom', fontsize=10)

# Training SNR distribution
ax = axes[1, 0]
unique_snr_train, counts_snr_train = np.unique(train_snr, return_counts=True)
ax.bar(unique_snr_train, counts_snr_train, color='steelblue', width=3)
ax.set_xlabel('SNR (dB)')
ax.set_ylabel('Count')
ax.set_title('Training Data: SNR Distribution')
ax.set_xticks(unique_snr_train)

# Test SNR distribution
ax = axes[1, 1]
unique_snr_test, counts_snr_test = np.unique(test_snr, return_counts=True)
ax.bar(unique_snr_test, counts_snr_test, color='steelblue', width=3)
ax.set_xlabel('SNR (dB)')
ax.set_ylabel('Count')
ax.set_title('Test Data: SNR Distribution')
ax.set_xticks(unique_snr_test)

plt.tight_layout()
plt.show()

# Print summary
print("\nTraining classes:", dict(zip(unique_train, counts_train)))
print("Test classes:", dict(zip(unique_test, counts_test)))
print("\nTraining SNR:", dict(zip(unique_snr_train, counts_snr_train)))
print("Test SNR:", dict(zip(unique_snr_test, counts_snr_test)))

In [ ]:
# Cross-tabulation: Class x SNR
print("\n" + "=" * 60)
print("TRAINING DATA: Class x SNR Cross-tabulation")
print("=" * 60)
train_df = pd.DataFrame({'class': train_labels, 'snr': train_snr})
print(pd.crosstab(train_df['class'], train_df['snr'], margins=True))

print("\n" + "=" * 60)
print("TEST DATA: Class x SNR Cross-tabulation")
print("=" * 60)
test_df = pd.DataFrame({'class': test_labels, 'snr': test_snr})
print(pd.crosstab(test_df['class'], test_df['snr'], margins=True))

## 3. Time Domain Visualization (IQ Signals)

In [ ]:
def plot_iq_signal(signal, label, snr, ax=None, title=None):
    """Plot I and Q components of a signal."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 4))
    
    t = np.arange(signal.shape[0])
    ax.plot(t, signal[:, 0], label='I (In-phase)', alpha=0.8, linewidth=0.5)
    ax.plot(t, signal[:, 1], label='Q (Quadrature)', alpha=0.8, linewidth=0.5)
    ax.set_xlabel('Sample')
    ax.set_ylabel('Amplitude')
    ax.legend(loc='upper right')
    if title:
        ax.set_title(title)
    else:
        ax.set_title(f'Class {label}, SNR={snr} dB')
    return ax

In [ ]:
# Plot one example from each class (training data, high SNR)
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()

for class_idx in range(6):
    # Get a high SNR example
    mask = (train_labels == class_idx) & (train_snr == 30)
    idx = np.where(mask)[0][0]
    plot_iq_signal(train_signals[idx], train_labels[idx], train_snr[idx], axes[class_idx])

plt.suptitle('Training Classes 0-5 (SNR=30 dB)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Plot anomaly classes from test set
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, class_idx in enumerate([6, 7, 8]):
    mask = (test_labels == class_idx) & (test_snr == 30)
    idx = np.where(mask)[0][0]
    plot_iq_signal(test_signals[idx], test_labels[idx], test_snr[idx], axes[i])
    axes[i].set_title(f'ANOMALY Class {class_idx}, SNR=30 dB', color='red')

plt.suptitle('Anomaly Classes 6-8 (SNR=30 dB)', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# Effect of SNR on the same class
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
axes = axes.flatten()

class_to_show = 0
snr_values = [30, 20, 10, 0]

for i, snr_val in enumerate(snr_values):
    mask = (train_labels == class_to_show) & (train_snr == snr_val)
    idx = np.where(mask)[0][0]
    plot_iq_signal(train_signals[idx], train_labels[idx], train_snr[idx], axes[i])

plt.suptitle(f'Class {class_to_show} at Different SNR Levels', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 4. Complex Signal Analysis (IQ Plane)

In [ ]:
def plot_iq_constellation(signal, ax=None, title=None, alpha=0.3, s=1):
    """Plot IQ constellation diagram."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    
    ax.scatter(signal[:, 0], signal[:, 1], alpha=alpha, s=s)
    ax.set_xlabel('I (In-phase)')
    ax.set_ylabel('Q (Quadrature)')
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
    ax.set_aspect('equal')
    if title:
        ax.set_title(title)
    return ax

In [ ]:
# IQ constellation for all training classes (high SNR)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for class_idx in range(6):
    mask = (train_labels == class_idx) & (train_snr == 30)
    idx = np.where(mask)[0][0]
    plot_iq_constellation(train_signals[idx], axes[class_idx], 
                         title=f'Class {class_idx} (SNR=30 dB)')

plt.suptitle('IQ Constellation Diagrams - Training Classes', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# IQ constellation for anomaly classes
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, class_idx in enumerate([6, 7, 8]):
    mask = (test_labels == class_idx) & (test_snr == 30)
    idx = np.where(mask)[0][0]
    plot_iq_constellation(test_signals[idx], axes[i], 
                         title=f'ANOMALY Class {class_idx} (SNR=30 dB)')

plt.suptitle('IQ Constellation Diagrams - Anomaly Classes', fontsize=14, y=1.05, color='red')
plt.tight_layout()
plt.show()

## 5. Frequency Domain Analysis

In [ ]:
def compute_complex_signal(signal):
    """Convert IQ to complex signal."""
    return signal[:, 0] + 1j * signal[:, 1]

def plot_spectrum(signal, ax=None, title=None, fs=1.0):
    """Plot power spectrum of complex signal."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 4))
    
    complex_sig = compute_complex_signal(signal)
    n = len(complex_sig)
    
    # Compute FFT
    spectrum = fftshift(fft(complex_sig))
    freqs = fftshift(fftfreq(n, 1/fs))
    power = 10 * np.log10(np.abs(spectrum)**2 + 1e-10)  # Power in dB
    
    ax.plot(freqs, power, linewidth=0.5)
    ax.set_xlabel('Normalized Frequency')
    ax.set_ylabel('Power (dB)')
    if title:
        ax.set_title(title)
    return ax

In [ ]:
# Power spectrum for all training classes
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()

for class_idx in range(6):
    mask = (train_labels == class_idx) & (train_snr == 30)
    idx = np.where(mask)[0][0]
    plot_spectrum(train_signals[idx], axes[class_idx], 
                 title=f'Class {class_idx} (SNR=30 dB)')

plt.suptitle('Power Spectra - Training Classes', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Power spectrum for anomaly classes
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, class_idx in enumerate([6, 7, 8]):
    mask = (test_labels == class_idx) & (test_snr == 30)
    idx = np.where(mask)[0][0]
    plot_spectrum(test_signals[idx], axes[i], 
                 title=f'ANOMALY Class {class_idx} (SNR=30 dB)')

plt.suptitle('Power Spectra - Anomaly Classes', fontsize=14, y=1.08, color='red')
plt.tight_layout()
plt.show()

In [ ]:
def plot_spectrogram(signal, ax=None, title=None, nperseg=128):
    """Plot spectrogram of complex signal."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 4))
    
    complex_sig = compute_complex_signal(signal)
    
    f, t, Sxx = scipy_signal.spectrogram(complex_sig, nperseg=nperseg, 
                                          noverlap=nperseg//2, return_onesided=False)
    # Shift frequencies to center
    Sxx = np.fft.fftshift(Sxx, axes=0)
    f = np.fft.fftshift(f)
    
    im = ax.pcolormesh(t, f, 10*np.log10(Sxx + 1e-10), shading='auto', cmap='viridis')
    ax.set_ylabel('Normalized Frequency')
    ax.set_xlabel('Time (samples)')
    if title:
        ax.set_title(title)
    return ax, im

In [ ]:
# Spectrograms for all training classes
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for class_idx in range(6):
    mask = (train_labels == class_idx) & (train_snr == 30)
    idx = np.where(mask)[0][0]
    plot_spectrogram(train_signals[idx], axes[class_idx], 
                    title=f'Class {class_idx} (SNR=30 dB)')

plt.suptitle('Spectrograms - Training Classes', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Spectrograms for anomaly classes
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, class_idx in enumerate([6, 7, 8]):
    mask = (test_labels == class_idx) & (test_snr == 30)
    idx = np.where(mask)[0][0]
    plot_spectrogram(test_signals[idx], axes[i], 
                    title=f'ANOMALY Class {class_idx} (SNR=30 dB)')

plt.suptitle('Spectrograms - Anomaly Classes', fontsize=14, y=1.08, color='red')
plt.tight_layout()
plt.show()

## 6. Statistical Features per Class

In [ ]:
def compute_signal_features(signal):
    """Compute various features from a signal."""
    complex_sig = compute_complex_signal(signal)
    
    # Amplitude (envelope)
    amplitude = np.abs(complex_sig)
    
    # Phase
    phase = np.angle(complex_sig)
    
    # Instantaneous frequency (phase derivative)
    inst_freq = np.diff(np.unwrap(phase))
    
    # Power
    power = np.abs(complex_sig)**2
    
    features = {
        'mean_amplitude': np.mean(amplitude),
        'std_amplitude': np.std(amplitude),
        'max_amplitude': np.max(amplitude),
        'mean_power': np.mean(power),
        'std_power': np.std(power),
        'mean_inst_freq': np.mean(inst_freq),
        'std_inst_freq': np.std(inst_freq),
        'i_mean': np.mean(signal[:, 0]),
        'i_std': np.std(signal[:, 0]),
        'q_mean': np.mean(signal[:, 1]),
        'q_std': np.std(signal[:, 1]),
        'iq_correlation': np.corrcoef(signal[:, 0], signal[:, 1])[0, 1],
        'kurtosis_i': np.mean((signal[:, 0] - signal[:, 0].mean())**4) / (signal[:, 0].std()**4 + 1e-10),
        'kurtosis_q': np.mean((signal[:, 1] - signal[:, 1].mean())**4) / (signal[:, 1].std()**4 + 1e-10),
    }
    return features

In [ ]:
# Compute features for all signals (this may take a moment)
print("Computing features for training data...")
train_features = [compute_signal_features(s) for s in train_signals]
train_features_df = pd.DataFrame(train_features)
train_features_df['class'] = train_labels
train_features_df['snr'] = train_snr
train_features_df['is_anomaly'] = False

print("Computing features for test data...")
test_features = [compute_signal_features(s) for s in test_signals]
test_features_df = pd.DataFrame(test_features)
test_features_df['class'] = test_labels
test_features_df['snr'] = test_snr
test_features_df['is_anomaly'] = test_labels >= 6

print("Done!")
print(f"\nTraining features shape: {train_features_df.shape}")
print(f"Test features shape: {test_features_df.shape}")

In [ ]:
# Summary statistics by class
print("\n" + "=" * 60)
print("TRAINING DATA: Mean Features by Class")
print("=" * 60)
print(train_features_df.groupby('class')[['mean_amplitude', 'std_amplitude', 'mean_power', 'std_inst_freq']].mean().round(4))

In [ ]:
# Summary statistics by class for test
print("\n" + "=" * 60)
print("TEST DATA: Mean Features by Class (including anomalies)")
print("=" * 60)
print(test_features_df.groupby('class')[['mean_amplitude', 'std_amplitude', 'mean_power', 'std_inst_freq']].mean().round(4))

In [ ]:
# Visualize feature distributions: Known vs Anomaly
features_to_plot = ['mean_amplitude', 'std_amplitude', 'mean_power', 'std_inst_freq', 'iq_correlation', 'kurtosis_i']

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, feat in enumerate(features_to_plot):
    ax = axes[i]
    
    # Known classes from test
    known = test_features_df[~test_features_df['is_anomaly']][feat]
    anomaly = test_features_df[test_features_df['is_anomaly']][feat]
    
    ax.hist(known, bins=50, alpha=0.6, label='Known (0-5)', color=KNOWN_COLOR, density=True)
    ax.hist(anomaly, bins=50, alpha=0.6, label='Anomaly (6-8)', color=ANOMALY_COLOR, density=True)
    ax.set_xlabel(feat)
    ax.set_ylabel('Density')
    ax.legend()
    ax.set_title(f'{feat} Distribution')

plt.suptitle('Feature Distributions: Known vs Anomaly Classes (Test Set)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 7. 2D Feature Scatter Plots

In [ ]:
# 2D scatter plots of features
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Mean amplitude vs std amplitude
ax = axes[0, 0]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    ax.scatter(test_features_df[mask]['mean_amplitude'], 
              test_features_df[mask]['std_amplitude'],
              c=[color], label=f'Class {class_idx}', alpha=0.5, marker=marker, s=20)
ax.set_xlabel('Mean Amplitude')
ax.set_ylabel('Std Amplitude')
ax.set_title('Mean vs Std Amplitude')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot 2: Mean power vs std instantaneous frequency
ax = axes[0, 1]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    ax.scatter(test_features_df[mask]['mean_power'], 
              test_features_df[mask]['std_inst_freq'],
              c=[color], label=f'Class {class_idx}', alpha=0.5, marker=marker, s=20)
ax.set_xlabel('Mean Power')
ax.set_ylabel('Std Instantaneous Frequency')
ax.set_title('Mean Power vs Std Inst. Frequency')

# Plot 3: I std vs Q std
ax = axes[1, 0]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    ax.scatter(test_features_df[mask]['i_std'], 
              test_features_df[mask]['q_std'],
              c=[color], label=f'Class {class_idx}', alpha=0.5, marker=marker, s=20)
ax.set_xlabel('I Std')
ax.set_ylabel('Q Std')
ax.set_title('I vs Q Standard Deviation')

# Plot 4: Kurtosis
ax = axes[1, 1]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    ax.scatter(test_features_df[mask]['kurtosis_i'], 
              test_features_df[mask]['kurtosis_q'],
              c=[color], label=f'Class {class_idx}', alpha=0.5, marker=marker, s=20)
ax.set_xlabel('Kurtosis I')
ax.set_ylabel('Kurtosis Q')
ax.set_title('I vs Q Kurtosis')

plt.tight_layout()
plt.show()

## 8. SNR Impact Analysis

In [ ]:
# How features change with SNR
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

features_vs_snr = ['mean_amplitude', 'std_amplitude', 'mean_power', 'std_inst_freq']

for i, feat in enumerate(features_vs_snr):
    ax = axes.flatten()[i]
    
    # Group by class and SNR
    for class_idx in range(6):  # Training classes only
        class_data = train_features_df[train_features_df['class'] == class_idx]
        means = class_data.groupby('snr')[feat].mean()
        stds = class_data.groupby('snr')[feat].std()
        ax.errorbar(means.index, means.values, yerr=stds.values, 
                   label=f'Class {class_idx}', marker='o', capsize=3)
    
    ax.set_xlabel('SNR (dB)')
    ax.set_ylabel(feat)
    ax.set_title(f'{feat} vs SNR')
    ax.legend()
    ax.set_xticks([0, 10, 20, 30])

plt.suptitle('Feature Variation with SNR (Training Classes)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 9. Correlation Analysis

In [ ]:
# Feature correlation heatmap
feature_cols = ['mean_amplitude', 'std_amplitude', 'max_amplitude', 'mean_power', 
                'std_power', 'mean_inst_freq', 'std_inst_freq', 'i_mean', 'i_std',
                'q_mean', 'q_std', 'iq_correlation', 'kurtosis_i', 'kurtosis_q']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training data correlation
corr_train = train_features_df[feature_cols].corr()
sns.heatmap(corr_train, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
           ax=axes[0], annot_kws={'size': 7})
axes[0].set_title('Feature Correlation - Training Data')

# Test data correlation
corr_test = test_features_df[feature_cols].corr()
sns.heatmap(corr_test, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
           ax=axes[1], annot_kws={'size': 7})
axes[1].set_title('Feature Correlation - Test Data')

plt.tight_layout()
plt.show()

## 10. PCA Visualization

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Standardize features
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_features_df[feature_cols])
test_scaled = scaler.transform(test_features_df[feature_cols])

# PCA
pca = PCA(n_components=2)
train_pca = pca.fit_transform(train_scaled)
test_pca = pca.transform(test_scaled)

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {sum(pca.explained_variance_ratio_):.2%}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training data PCA
ax = axes[0]
for class_idx in range(6):
    mask = train_features_df['class'] == class_idx
    ax.scatter(train_pca[mask, 0], train_pca[mask, 1], 
              c=[CLASS_COLORS[class_idx]], label=f'Class {class_idx}', alpha=0.5, s=10)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
ax.set_title('PCA - Training Data (Known Classes)')
ax.legend()

# Test data PCA with anomalies highlighted
ax = axes[1]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    label = f'ANOMALY {class_idx}' if class_idx >= 6 else f'Class {class_idx}'
    ax.scatter(test_pca[mask, 0], test_pca[mask, 1], 
              c=[color], label=label, alpha=0.6, marker=marker, s=20)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
ax.set_title('PCA - Test Data (Known + Anomalies)')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 11. t-SNE Visualization

In [ ]:
from sklearn.manifold import TSNE

# Subsample for faster t-SNE (use all test data + subset of train)
np.random.seed(42)
train_sample_idx = np.random.choice(len(train_scaled), size=min(2000, len(train_scaled)), replace=False)
train_sample = train_scaled[train_sample_idx]
train_sample_labels = train_features_df['class'].values[train_sample_idx]

print("Running t-SNE on training sample...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
train_tsne = tsne.fit_transform(train_sample)

print("Running t-SNE on test data...")
tsne_test = TSNE(n_components=2, perplexity=30, random_state=42)
test_tsne = tsne_test.fit_transform(test_scaled)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training data t-SNE
ax = axes[0]
for class_idx in range(6):
    mask = train_sample_labels == class_idx
    ax.scatter(train_tsne[mask, 0], train_tsne[mask, 1], 
              c=[CLASS_COLORS[class_idx]], label=f'Class {class_idx}', alpha=0.5, s=10)
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('t-SNE - Training Data (Known Classes)')
ax.legend()

# Test data t-SNE with anomalies highlighted
ax = axes[1]
for class_idx in range(9):
    mask = test_features_df['class'] == class_idx
    color = ANOMALY_COLOR if class_idx >= 6 else CLASS_COLORS[class_idx]
    marker = 'x' if class_idx >= 6 else 'o'
    label = f'ANOMALY {class_idx}' if class_idx >= 6 else f'Class {class_idx}'
    ax.scatter(test_tsne[mask, 0], test_tsne[mask, 1], 
              c=[color], label=label, alpha=0.6, marker=marker, s=20)
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('t-SNE - Test Data (Known + Anomalies)')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 12. Key Observations Summary

In [ ]:
print("=" * 70)
print("KEY OBSERVATIONS FROM EXPLORATORY DATA ANALYSIS")
print("=" * 70)

print("\n1. DATASET STRUCTURE:")
print(f"   - Training: {len(train_signals)} samples, classes 0-5, SNR [0, 10, 20, 30] dB")
print(f"   - Test: {len(test_signals)} samples, classes 0-8, SNR [10, 20, 30] dB")
print(f"   - Signal shape: 2048 time samples x 2 channels (I/Q)")
print(f"   - NOTE: Training has SNR=0dB but test does NOT")

print("\n2. CLASS BALANCE:")
print(f"   - Training: ~{counts_train[0]} samples per class (balanced)")
print(f"   - Test known: {sum(counts_test[:6])} samples from classes 0-5")
print(f"   - Test anomalies: {sum(counts_test[6:])} samples from classes 6-8")

print("\n3. SIGNAL CHARACTERISTICS:")
print(f"   - Different modulation patterns visible in time domain")
print(f"   - IQ constellation shows different modulation schemes")
print(f"   - Distinct spectral signatures per class")

print("\n4. SNR IMPACT:")
print(f"   - Lower SNR increases noise, making classification harder")
print(f"   - Signal features are SNR-dependent")
print(f"   - Anomaly detection should account for SNR variation")

print("\n5. SEPARABILITY (from PCA/t-SNE):")
print(f"   - Classes show some clustering in feature space")
print(f"   - Anomaly classes may or may not overlap with known classes")
print(f"   - Simple features may provide baseline separation")

print("\n6. IMPLICATIONS FOR ANOMALY DETECTION:")
print("   - Need to learn 'normal' distribution of classes 0-5")
print("   - Features like amplitude, power, inst. freq. could help")
print("   - Deep learning can learn better representations")
print("   - Consider SNR-aware normalization/conditioning")